In [2]:
# all imports
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle
import pandas as pd
import random
from matplotlib.patches import Ellipse
import ligo.skymap.plot
from scipy.optimize import minimize
import numdifftools as ndt

In [28]:
#constants
G = 4.30091e-6  # gravitational constant in kpc (km/s)^2 / Msun 
c = 299792.458  #km/s
H = 100

In [29]:
# define directories
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/j/jjpim/cache/' # my cache path
v_dir = '/pscratch/sd/j/jjpim/rotcurves/'

# load updated master table
tf_mastertable = Table.read(data_dir + 'tf_mastertable.fits') 

#unique SGA IDs in rot curve galaxies
tf_mastertable_dict = {}
for sga_id in np.unique(tf_mastertable['SGA_ID'][tf_mastertable['ROT_CURVE']==1]):
    tf_mastertable_dict[sga_id] = np.where(tf_mastertable['SGA_ID'] == sga_id)[0]

# load SGA table
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')
SGA_dict = {}
for i in range(len(SGA)):
    SGA_dict[SGA['SGA_ID'][i]] = i

In [43]:
SGA[SGA['SGA_ID']== 31220]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
31220,SGA-2020 31220,PGC058127,58127,246.6315765,44.1220467,Sbc,11.6,0.57942873,0.75335556,0.031340346,23.70848,16.265,False,LEDA-20181114,8361,PGC058123_GROUP,2,False,246.6218172418136,44.11696306357726,1.1335149,2466p440,246.6315996119623,44.122057654369335,0.7242859,LEDA,11.22325,0.80032957,246.63140681000476,44.12209227169959,22.152418,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0


In [46]:
tf_mastertable[tf_mastertable['SGA_ID']==1001302]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VERR,Z_CENTER,ROTCURVE_CHI2,VMAX,VMAX_ERR,RTURN,RTURN_ERR,ALPHA,ALPHA_ERR,INCLINATION,VROT_R26,VROT_R26_ERR,M_R26,M_R26_ERR,VROT_3P5R26,VROT_3P5R26_ERR,M_3P5R26,M_3P5R26_ERR,RABS_MAG26
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1070132092731403,179.470211844291,0.8587724859608,27650,sv3,0.105622028293016,1.44051968704578e-05,0,103.813973851502,fuji/healpix/sv3/bright/276/27650/redrock-sv3-bright-27650.fits,TFT,1001302,179.47021184429076,0.8587724859608005,1.0,0.0025639894776274498,0.6700000103246535,1.0,0.0,-75.38931786886482,4.393662659673312,0.10587349998904133,588.4038085786137,147.4872219372602,0.8579026510102183,0.30960505249344444,0.06259765002020086,99.9837936498326,272.1652990680309,1.2759389754248802,147.48722193726022,12.292899522662395,11.030275199956364,0.06695838807163135,147.48722193726022,17.314528051152024,11.57434324430664,0.09159437058325004,-5.122854624529303
1083326232264715,179.470211844291,0.8587724859608,27650,sv3,0.105649103365147,1.37439786192377e-05,0,251.056394457817,fuji/healpix/sv3/dark/276/27650/redrock-sv3-dark-27650.fits,TFT,1001302,179.47021184429076,0.8587724859608005,1.0,0.0025639894776274498,0.6700000103246535,1.0,0.0,-67.27241544418176,4.198980965469619,0.10587349998904133,588.4038085786137,147.4872219372602,0.8579026510102183,0.30960505249344444,0.06259765002020086,99.9837936498326,272.1652990680309,1.2759389754248802,147.48722193726022,12.292899522662395,11.030275199956364,0.06695838807163135,147.48722193726022,17.314528051152024,11.57434324430664,0.09159437058325004,-5.122854624529303
1083326232264713,179.466571130357,0.857592250335877,27650,sv3,0.106362597976074,3.00124771947937e-06,0,3871.25708341599,fuji/healpix/sv3/dark/276/27650/redrock-sv3-dark-27650.fits,TFT,1001302,179.4665711303573,0.8575922503358774,1.0,0.0012628604889797456,0.3300000050850318,1.0,0.0,146.62788773537676,1.2098688911597317,0.10587349998904133,588.4038085786137,147.4872219372602,0.8579026510102183,0.30960505249344444,0.06259765002020086,99.9837936498326,272.1652990680309,1.2759389754248802,147.48722193726022,12.292899522662395,11.030275199956364,0.06695838807163135,147.48722193726022,17.314528051152024,11.57434324430664,0.09159437058325004,-5.122854624529303
1070132092731399,179.464131853278,0.856801490529996,27650,sv3,1.03087178074219,0.000197813829449578,4,1.33134818449616,fuji/healpix/sv3/bright/276/27650/redrock-sv3-bright-27650.fits,TFT,1001302,179.46413185327816,0.856801490529996,1.0,0.0038268499665838206,1.0000000154035773,1.0,0.0,277307.50823276053,59.30860988676193,0.10587349998904133,--,--,--,--,--,--,--,--,nan,0.0,1.0,1.0,nan,0.0,1.0,1.0,-5.122854624529303
39627805855585329,179.467772565707,0.857981728475074,27650,sv3,0.1058810661205,5.16938798206203e-06,0,3628.18237131834,fuji/healpix/sv3/bright/276/27650/redrock-sv3-bright-27650.fits,TFT,1001302,179.46777256570698,0.8579817284750743,1.0,9.93923337957349e-17,2.5972362698989904e-14,1.0,0.0,2.2682691475439807,1.748123309045789,0.10587349998904133,588.4038085786137,147.4872219372602,0.8579026510102183,0.30960505249344444,0.06259765002020086,99.9837936498326,272.1652990680309,1.2759389754248802,147.48722193726022,12.292899522662395,11.030275199956364,0.06695838807163135,147.48722193726022,17.314528051152024,11.57434324430664,0.09159437058325004,-5.122854624529303
1070132092731404,179.471413280382,0.859161962956448,27650,sv3,1.52278831950645,0.000119143196848319,4,0.78112968057394,fuji/healpix/sv3/bright/276/27650/redrock-sv3-bright-27650.fits,TFT,1001302,179.47141328038205,0.859161962956448,1.0,0.0038268499666231693,1.0000000154138595,1.0,

In [45]:
SGA[SGA['SGA_ID']== 1001302]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
1001302,SGA-2020 1001302,PGC1177421,1177421,179.4677715,0.8579886,--,69.64,0.41783038,0.3548134,0.10590726,24.72748,17.994,False,LEDA-20181114,367770,PGC1177421,1,True,179.4677715,0.8579886,0.41783038,1793p007,179.46777256570698,0.8579817284750741,0.459222,SB26,72.036606,0.34795445,179.46754570859778,0.8579873275210212,11.05459,5.6530657,5.104197,4.99113,3.6659405,5.297342,6.8670187,8.303286,9.554755,10.607548,11.479372,12.513401,13.77666,19.135197,18.514793,18.055344,18.610353,18.046705,17.617245,18.295559,17.777506,17.36816,18.09619,17.600636,17.206568,17.980507,17.501755,17.113266,17.915882,17.449116,17.06284,17.880283,17.420956,17.03312,17.852493,17.401283,17.007994,17.831041,17.385185,16.991564,0.04452628,0.06684575,0.06077771,0.06216872,0.06596702,0.06663131,0.05344319,0.082110435,0.19219008,0.030917166,0.037118454,0.039950326,0.026310228,0.030867003,0.03279852,0.023930356,0.027698016,0.028770646,0.0223635,0.025262905,0.026193593,0.021120043,0.023711866,0.024591547,0.02018419,0.022715302,0.023612978,0.019654883,0.0221756,0.023063712,0.019225596,0.021832649,0.022614298,0.018892244,0.021577386,0.022342043,17.779795,0.31523618,0.72515684,4.579929,0.22905691,17.350004,0.25832015,0.64711636,4.8961196,0.30607206,16.940527,0.29201382,0.67604846,4.158233,0.20092589,0


In [32]:
sga_ids = [9769, 11977, 12892, 14501, 16383, 19235, 19278, 25532, 30149, 31220, 32484, 36904, 37543, 46327, 58617, 73111, 88963, 91660, 90791, 106536, 
           114405, 114489, 122580, 126630, 125611, 128163, 128411, 130766, 149502, 153504, 175220, 176736, 184044, 186239, 189685, 188525, 190308, 
           192158, 196470, 199038, 202236, 205028, 209429, 212382, 218773, 235556, 254532, 272164, 272562, 276638, 288715, 289743, 291766, 300446, 
           302062, 305831, 309378, 319942, 330678, 349027, 349346, 363344, 373692, 378842, 384420, 389177, 393190, 395070, 398443, 398532, 416531, 
           431242, 430161, 432294, 450689, 450566, 455486, 457147, 458798, 462867, 466872, 467582, 471327, 469868, 471893, 473436, 477805, 481956, 
           485800, 493017, 507499, 526710, 544419, 544810, 544799, 544943, 548724, 548848, 550458, 564440, 569298, 568664, 570086, 582248, 591927, 
           607528, 608015, 629144, 630029, 630390, 635043, 638386, 641578, 643231, 643845, 644940, 647966, 649753, 649960, 651096, 654469, 659599, 
           663463, 671301, 671533, 694525, 696291, 696345, 706579, 720480, 722142, 726854, 728327, 729726, 729754, 736449, 742167, 743030, 744650, 
           752683, 757313, 757166, 758123, 759261, 759003, 766672, 783693, 793633, 804643, 805225, 819754, 820598, 823301, 824225, 826809, 827949, 
           833961, 833981, 843065, 843126, 845052, 887433, 901663, 901980, 908818, 908978, 911046, 914975, 914909, 916161, 920806, 923400, 925923, 
           925354, 928810, 928876, 932984, 947709, 953644, 957521, 957850, 967351, 968864, 971635, 975775, 972260, 977015, 976853, 979168, 978708, 
           989040, 989244, 1001302, 1003319, 1008726, 1014959, 1018187, 1021462, 1026141, 1031898, 1033651, 1033344, 1050173, 1065602, 1073430, 
           1086934, 1087039, 1087374, 1090772, 1091806, 1096564, 1098043, 1103477, 1111824, 1117018, 1124386, 1143335, 1146354, 1161160, 1167691, 
           1171841, 1179377, 1184894, 1184912, 1186034, 1187240, 1188032, 1188617, 1188926, 1193015, 1191665, 1194588, 1194958, 1199020, 1199539, 
           1201319, 1203786, 1204237, 1206707, 1218166, 1219319, 1254154, 1264056, 1264591, 1267967, 1271477, 1271549, 1272700, 1273605, 1278144, 
           1281619, 1282407, 1285273, 1286710, 1289613, 1293967, 1301768, 1304908, 1335916, 1338866, 1345100, 1348956, 1349555, 1354266, 1355575, 
           1361363, 1361399, 1365519, 1365549, 1367137, 1368018, 1368443, 1374964, 1379143, 1405053, 1429680, 1434487]

In [33]:
#R26 columns
tf_mastertable['VROT_R26']=np.nan
tf_mastertable['VROT_R26_ERR']=np.nan
tf_mastertable['M_R26']=np.nan
tf_mastertable['M_R26_ERR']=np.nan
#3.5 R26 columns
tf_mastertable['VROT_3P5R26']=np.nan
tf_mastertable['VROT_3P5R26_ERR']=np.nan
tf_mastertable['M_3P5R26']=np.nan
tf_mastertable['M_3P5R26_ERR']=np.nan
#Absolute Magnitude Column
tf_mastertable['RABS_MAG26']=np.nan

In [34]:
#functions for vrot and mass w/ errors calculations
def v_rot(r, v_max, r_turn, alpha):
    '''
    Calculate Rotational Velocity at a radius
    Parameters
    ----------
    r : float
        Target distance.
    v_max : float
        Maximum velocity.
    r_turn: float
        Velocity curve turning point.
    alpha : 
        Velocity curve angle.
    Returns
    -------
    Rotational velocity in km / s
    '''
    vrot = (v_max * r) / ((r_turn ** alpha + r ** alpha) ** (1/alpha))
    return vrot

#vrot error calculations
def v_rot_error(r, v_max, r_turn, alpha, v_max_err, r_turn_err, alpha_err):
    '''
    Calculate error in Rotational Velocity given errors in parameters
    Parameters
    ----------
    r : float
        Radius in kpc.
    v_max : float
        Maximum velocity.
    r_turn: float
        Velocity curve turning point.
    alpha : float
        Velocity curve angle.
    v_max_err : float
        Uncertainty in maximum velocity.
    r_turn_err : float
        Uncertainty in velocity curve turning point.
    alpha_err : float
        Uncertainty in velocity curve angle.
    Returns
    -------
    Error in rotational velocity in km / s
    '''
    # Calculate the partial derivatives
    common_denom = (r_turn ** alpha + r ** alpha)
    dvrot_dvmax = r / (common_denom ** (1 / alpha))
    dvrot_drturn = -v_max * r * (r_turn ** (alpha - 1)) * (np.log(common_denom)) / (common_denom ** ((1 / alpha) + 1))
    dvrot_dalpha = -v_max * r * (r_turn ** alpha * np.log(r_turn) + r ** alpha * np.log(r)) / (alpha ** 2 * common_denom ** ((1 / alpha) + 1))

    # Calculate the error propagation
    v_rot_err = np.sqrt((dvrot_dvmax * v_max_err) ** 2 +
                        (dvrot_drturn * r_turn_err) ** 2 +
                        (dvrot_dalpha * alpha_err) ** 2)
    
    return v_rot_err

def chi_2(params, v, v_err, r):
    '''
    Calculate Chi Squared 
    Parameters
    ----------
    params : list
        v_max, r_turn, alpha.
    v : float
        Absolute value of velocity.
    v_err : float
        Uncertainty for velocity.
    r : 
        Target distance.
    Returns
    -------
    Chi Squared value
    '''
    v_max, r_turn, alpha = params
    v_model = v_rot(r, v_max, r_turn, alpha)
    chi_sq = np.sum((v_model - v)**2/(v_err**2))
    return chi_sq

def mass_calc(v_rot, r, v_rot_err):
    '''
    Calculate mass in log10 solar masses
    Parameters
    ----------
    v_rot : float
        Rotational velocity in km / s.
    r : float
        radius in kpc.
    Returns
    -------
    Mass in log10 solar masses
    '''
    #mass calculation
    mass = (v_rot**2 * r) / G
    
    #calculate mass error
    mass_err = mass * np.sqrt((2 * v_rot_err / v_rot)**2)

    # log of mass and its error
    log_mass = np.log10(mass)
    log_mass_err = np.log10(mass + mass_err) - log_mass

    return log_mass, log_mass_err

#function for magnitude calculations
def mag_calc(app_mag, d):
    '''
    Calculate Absoulte Magnitude
    Parameters
    ----------
    app_mag : float
        Apparent Magnitude.
    d : float
        Distance.
    Returns
    -------
    Absolute Magnitude
    '''
    abs_mag = app_mag - 5*np.log10(d/10)
    return abs_mag

In [35]:
#calculate R26 mass for all galaxies w/ errors
for sga_id in sga_ids:
    sga = SGA[SGA['SGA_ID']==sga_id]
    galaxy = tf_mastertable[tf_mastertable['SGA_ID']==sga_id]
    redshift = galaxy['Z_CENTER']
    z = np.abs(redshift)

    #parameters
    diam = 0.5*sga['D26'] #arcmin
    theta = diam * (np.pi / (180*60)) #rads
    d = 1000*((c*z)/H) #kpcs
    r = d * np.tan(theta) #kpc
    v_max = galaxy['VMAX']
    v_max_err = galaxy['VMAX_ERR']
    r_turn = galaxy['RTURN']
    r_turn_err = galaxy['RTURN_ERR']
    alpha = galaxy['ALPHA']
    alpha_err = galaxy['ALPHA_ERR']
                                            
    # calculate the vrot at R26
    vrot = v_rot(r, v_max, r_turn, alpha)

    # calculate the vrot error
    vrot_err = v_rot_error(r, v_max, r_turn, alpha, v_max_err, r_turn_err, alpha_err)
    
    #add vrot and error to table
    tf_mastertable['VROT_R26'][tf_mastertable['SGA_ID']==sga_id] = vrot
    tf_mastertable['VROT_R26_ERR'][tf_mastertable['SGA_ID']==sga_id] = vrot_err
    
    # calculate the mass and mass error in log10 solar masses using vrot
    log_mass, log_mass_err = mass_calc(vrot, r, vrot_err)
    
    #adding mass and error to table
    tf_mastertable['M_R26'][tf_mastertable['SGA_ID']==sga_id] = log_mass
    tf_mastertable['M_R26_ERR'][tf_mastertable['SGA_ID']==sga_id] = log_mass_err

In [36]:
#calculate 3.5R26 mass for all galaxies w/ errors
for sga_id in sga_ids:
    sga = SGA[SGA['SGA_ID']==sga_id]
    galaxy = tf_mastertable[tf_mastertable['SGA_ID']==sga_id]
    redshift = galaxy['Z_CENTER']
    z = np.abs(redshift)

    #parameters
    diam = 0.5*sga['D26'] #arcmin
    theta = diam * (np.pi / (180*60)) #rads
    d = 1000*((c*z)/H) #kpcs
    r = 3.5 * d * np.tan(theta) #kpc
    v_max = galaxy['VMAX']
    v_max_err = galaxy['VMAX_ERR']
    r_turn = galaxy['RTURN']
    r_turn_err = galaxy['RTURN_ERR']
    alpha = galaxy['ALPHA']
    alpha_err = galaxy['ALPHA_ERR']
    # calculate the vrot at R26
    vrot = v_rot(r, v_max, r_turn, alpha)

    # calculate the vrot error
    vrot_err = v_rot_error(r, v_max, r_turn, alpha, v_max_err, r_turn_err, alpha_err)
    
    #add vrot and error to table
    tf_mastertable['VROT_3P5R26'][tf_mastertable['SGA_ID']==sga_id] = vrot
    tf_mastertable['VROT_3P5R26_ERR'][tf_mastertable['SGA_ID']==sga_id] = vrot_err
    
    # calculate the mass and mass error in log10 solar masses using vrot
    log_mass, log_mass_err = mass_calc(vrot, r, vrot_err)
    
    #adding mass and error to table
    tf_mastertable['M_3P5R26'][tf_mastertable['SGA_ID']==sga_id] = log_mass
    tf_mastertable['M_3P5R26_ERR'][tf_mastertable['SGA_ID']==sga_id] = log_mass_err

In [47]:
for sga_id in sga_ids:
    sga = SGA[SGA['SGA_ID']==sga_id]
    galaxy = tf_mastertable[tf_mastertable['SGA_ID']==sga_id]
    app_mag = sga['R_MAG_SB26']
    if app_mag != -1:
        redshift = galaxy['Z_CENTER']
        z = np.abs(redshift)
        d_kpc = 1000*((c*z)/H) #kpcs
        d = (d_kpc)*1000
        abs_mag = mag_calc(app_mag, d)
        tf_mastertable['RABS_MAG26'][tf_mastertable['SGA_ID']==sga_id] = abs_mag

In [48]:
tf_mastertable[tf_mastertable['SGA_ID']==1001302]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VERR,Z_CENTER,ROTCURVE_CHI2,VMAX,VMAX_ERR,RTURN,RTURN_ERR,ALPHA,ALPHA_ERR,INCLINATION,VROT_R26,VROT_R26_ERR,M_R26,M_R26_ERR,VROT_3P5R26,VROT_3P5R26_ERR,M_3P5R26,M_3P5R26_ERR,RABS_MAG26
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1070132092731403,179.470211844291,0.8587724859608,27650,sv3,0.105622028293016,1.44051968704578e-05,0,103.813973851502,fuji/healpix/sv3/bright/276/27650/redrock-sv3-bright-27650.fits,TFT,1001302,179.47021184429076,0.8587724859608005,1.0,0.0025639894776274498,0.6700000103246535,1.0,0.0,-75.38931786886482,4.393662659673312,0.10587349998904133,588.4038085786137,147.4872219372602,0.8579026510102183,0.30960505249344444,0.06259765002020086,99.9837936498326,272.1652990680309,1.2759389754248802,147.48722193726022,12.292899522662395,11.030275199956364,0.06695838807163135,147.48722193726022,17.314528051152024,11.57434324430664,0.09159437058325004,-20.122854624529303
1083326232264715,179.470211844291,0.8587724859608,27650,sv3,0.105649103365147,1.37439786192377e-05,0,251.056394457817,fuji/healpix/sv3/dark/276/27650/redrock-sv3-dark-27650.fits,TFT,1001302,179.47021184429076,0.8587724859608005,1.0,0.0025639894776274498,0.6700000103246535,1.0,0.0,-67.27241544418176,4.198980965469619,0.10587349998904133,588.4038085786137,147.4872219372602,0.8579026510102183,0.30960505249344444,0.06259765002020086,99.9837936498326,272.1652990680309,1.2759389754248802,147.48722193726022,12.292899522662395,11.030275199956364,0.06695838807163135,147.48722193726022,17.314528051152024,11.57434324430664,0.09159437058325004,-20.122854624529303
1083326232264713,179.466571130357,0.857592250335877,27650,sv3,0.106362597976074,3.00124771947937e-06,0,3871.25708341599,fuji/healpix/sv3/dark/276/27650/redrock-sv3-dark-27650.fits,TFT,1001302,179.4665711303573,0.8575922503358774,1.0,0.0012628604889797456,0.3300000050850318,1.0,0.0,146.62788773537676,1.2098688911597317,0.10587349998904133,588.4038085786137,147.4872219372602,0.8579026510102183,0.30960505249344444,0.06259765002020086,99.9837936498326,272.1652990680309,1.2759389754248802,147.48722193726022,12.292899522662395,11.030275199956364,0.06695838807163135,147.48722193726022,17.314528051152024,11.57434324430664,0.09159437058325004,-20.122854624529303
1070132092731399,179.464131853278,0.856801490529996,27650,sv3,1.03087178074219,0.000197813829449578,4,1.33134818449616,fuji/healpix/sv3/bright/276/27650/redrock-sv3-bright-27650.fits,TFT,1001302,179.46413185327816,0.856801490529996,1.0,0.0038268499665838206,1.0000000154035773,1.0,0.0,277307.50823276053,59.30860988676193,0.10587349998904133,--,--,--,--,--,--,--,--,nan,0.0,1.0,1.0,nan,0.0,1.0,1.0,-20.122854624529303
39627805855585329,179.467772565707,0.857981728475074,27650,sv3,0.1058810661205,5.16938798206203e-06,0,3628.18237131834,fuji/healpix/sv3/bright/276/27650/redrock-sv3-bright-27650.fits,TFT,1001302,179.46777256570698,0.8579817284750743,1.0,9.93923337957349e-17,2.5972362698989904e-14,1.0,0.0,2.2682691475439807,1.748123309045789,0.10587349998904133,588.4038085786137,147.4872219372602,0.8579026510102183,0.30960505249344444,0.06259765002020086,99.9837936498326,272.1652990680309,1.2759389754248802,147.48722193726022,12.292899522662395,11.030275199956364,0.06695838807163135,147.48722193726022,17.314528051152024,11.57434324430664,0.09159437058325004,-20.122854624529303
1070132092731404,179.471413280382,0.859161962956448,27650,sv3,1.52278831950645,0.000119143196848319,4,0.78112968057394,fuji/healpix/sv3/bright/276/27650/redrock-sv3-bright-27650.fits,TFT,1001302,179.47141328038205,0.859161962956448,1.0,0.0038268499666231693,1.0000000154138595

In [55]:
tf_mastertable['M_R26'][tf_mastertable['M_R26']==1] = np.nan
tf_mastertable['M_R26_ERR'] [tf_mastertable['M_3P5R26_ERR']==1]= np.nan
tf_mastertable['M_3P5R26'][tf_mastertable['M_3P5R26_ERR']==1] = np.nan
tf_mastertable['M_3P5R26_ERR'][tf_mastertable['M_3P5R26_ERR']==1] = np.nan

In [56]:
tf_mastertable[tf_mastertable['M_3P5R26']==1]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VERR,Z_CENTER,ROTCURVE_CHI2,VMAX,VMAX_ERR,RTURN,RTURN_ERR,ALPHA,ALPHA_ERR,INCLINATION,VROT_R26,VROT_R26_ERR,M_R26,M_R26_ERR,VROT_3P5R26,VROT_3P5R26_ERR,M_3P5R26,M_3P5R26_ERR,RABS_MAG26
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


In [57]:
#writing new data into new table
tf_mastertable.write(data_dir + 'tf_mastertable_massmag.fits', format='fits', overwrite=True)